<a href="https://colab.research.google.com/github/GustavinhoDantas/LeituraPDFpy/blob/main/LeitorPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LEITOR DE PDF**
A ideia é pegar o PDF e ler de maneira automatica, onde o valor lidro ira retornar como arquivo excel ou SQL

In [11]:

!pip install pdfplumber pandas openpyxl

In [12]:
import pdfplumber
import re
import pandas as pd

# Lista para armazenar os dicionários dos dados extraídos
dados = []

# Função para formatação dos números
def format_nmr(nmr):
  if nmr != '':
    nmr = '(' + nmr[:2] + ')' + nmr[2:]
  return nmr

with pdfplumber.open('/content/Leo_Teste_PDF.pdf') as pdf:
    texto_completo = ''
    for pagina in pdf.pages:
        texto_completo += pagina.extract_text() + '\n'

    # Quebra o texto em blocos usando a linha cheia de pontos
    blocos = re.split(r'\n\.{10,}\n', texto_completo)


    for bloco in blocos:
     if 'Placa.......:' in bloco:
        # Tratamento especial para a Placa
        match = re.search(r'Placa\.*:\s*(.{7})', bloco)
        placa = match.group(1).strip() if match else ''

        # Tratamento especial para a Marca
        match = re.search(r'Marca\.*:\s*(.*)', bloco)
        marca = match.group(1).strip() if match else ''
        marca = re.sub(r'Ano Fab/Mod:.*', "", marca)

        # Tratamento especial para a Proprietário
        match = re.search(r'Prop. Atual\.*:\s*(.*)', bloco)
        proprietario = match.group(1).strip() if match else ''

        # Extrair telefones, só os 11 primeiros números
        match = re.search(r'Fone Res\.*:\s*(.{13})', bloco)
        telefone_res = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''
        telefone_res = format_nmr(telefone_res)

        match = re.search(r'Fone Com\.*:\s*(.{13})', bloco)
        telefone_com = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''
        telefone_com = format_nmr(telefone_com)

        match = re.search(r'Fone Cel\.*:\s*(.{13})', bloco)
        telefone_cel = ''.join(filter(str.isdigit, match.group(1)))[:11] if match else ''
        telefone_cel = format_nmr(telefone_cel)

        # Extrair origem sendo ele a linha toda da leitura
        match = re.search(r'Origem\.*:\s*(.*)', bloco)
        origem = match.group(1).strip() if match else ''
        if origem == 'Destinatário:':
          origem = ''

        numero_origem = re.sub(r'\D+', "", origem)

        # Extrair exercicios, sendo ele sempre quatro digitos após :
        match = re.search(r'Exer:\s*(.{4})', bloco)
        exercicio = match.group(1).strip() if match else ''
        if exercicio == 'Prop. Atual':
          exercicio = ''

        dados.append({
            'PLACA': placa,
            'MARCA': marca,
            'ORIGEM': origem,
            'NÚMERO DA ORIGEM': numero_origem,
            'PROPRIETÁRIO ATUAL': proprietario,
            'TELEFONE RESPONSÁVEL': telefone_res,
            'TELEFONE CELULAR': telefone_cel,
            'TELEFONE COMERCIAL': telefone_com,
            'EXERCÍCIO': exercicio,
        })
# Transforma em DataFrame e salva
df = pd.DataFrame(dados)
df.to_excel('PDFGLviews1.xlsx', index=False)
print('Dados extraídos e salvos no Excel!')

Dados extraídos e salvos no Excel!


#"Views do projeto"#


In [13]:
# Esqueleto dos filtros

#Separa tudo com esta origem
filtro_origem = df[df['ORIGEM'].isin(['SUSI COBRA E ENTREGA', 'DANIEL PAGA (TUTUCA)'])].copy()
filtro_origem.to_excel('Origens_separadas.xlsx', index=False)
print("Planília SUSI e TUTUCA separados")

#Pega todas as linhas em que tem origem e não tem número
filtro_garagens = df[
    (df['ORIGEM'] != "") &
    (df['TELEFONE CELULAR'] == "") &
    (df['TELEFONE COMERCIAL'] == "") &
    (df['TELEFONE RESPONSÁVEL'] == "")].copy()
filtro_garagens.to_excel('Garagens_separadas.xlsx', index=False)
print("Planília Garagens sem número separadas")
## FAZER uma planília separada, tantas placas em tal garagem, usar somente ['origem'], ['Prop atual'], ['placa']

#Pega todos os clientes sem contato "S/N"
filtro_contato = df[
    (df['ORIGEM'] == "") &
    (df['TELEFONE CELULAR'] == "") &
    (df['TELEFONE COMERCIAL'] == "") &
    (df['TELEFONE RESPONSÁVEL'] == "")].copy()
print("Planília Clientes sem contato separadas")
## Grifar no .xlsx todas as linhas

filtro_contato.to_excel('Contatos_separados.xlsx', index=False)

Planília SUSI e TUTUCA separados
Planília Garagens sem número separadas
Planília Clientes sem contato separadas


# MELHORIAS
*   interromper ciclo caso PDF corrompido
*   Item da lista
*   Adicionar os filtros no código principal na versão final


